# A4 Instacart Pt. II

# Contents
#### Import
#### Profiling Variables
-Create flag columns for use in customer profiles or direct analytics   
-Pet ownership (separate flag column from main profiles)  
-Customer profiles (age, income, # dependants)  
#### Create Customer Profiles
-Concatenate profile flags to form full profile variable  
-Cutoff (2 million orders) with profiles below being reassigned to "Default"  
-Exploratory descriptive stats on customer profiles  

## Import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats

In [2]:
# Our dataframe is pretty wide. 
pd.set_option("display.max_columns", None)

In [4]:
# Path variable to our project folder
path = r'D:\2021 CareerFoundry Course\Immersion\Instacart Basket Analysis A4'

In [5]:
# Read pickle file to df
ords_active_only = pd.read_pickle(os.path.join(path, '02 Data', 'Processed', 'ords_active_sample_cleaned.pkl'))

In [6]:
ords_active_only.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_user_last_ordered,Repeat_orders,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_hour,max_order,loyalty_flag,mean_price,spend_flag,median_freq,order_freq_flag,gender,state,age,date_joined,n_dependants,fam_status,income,_merge,region,exclusion_flag
0,2539329,1,1,2,8,NaN,False,196,1,0,Soda,77,7,9.0,Mid,Regularly busy,Regular days,Average,10,New customer,6.367188,Low,20.5,Low,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Active
1,2398795,1,2,3,7,15.0,True,196,1,1,Soda,77,7,9.0,Mid,Regularly busy,Slowest days,Average,10,New customer,6.367188,Low,20.5,Low,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Active
2,473747,1,3,3,12,21.0,True,196,1,1,Soda,77,7,9.0,Mid,Regularly busy,Slowest days,Busy,10,New customer,6.367188,Low,20.5,Low,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Active
3,2254736,1,4,4,7,29.0,True,196,1,1,Soda,77,7,9.0,Mid,Least busy,Slowest days,Average,10,New customer,6.367188,Low,20.5,Low,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Active
4,431534,1,5,4,15,28.0,True,196,1,1,Soda,77,7,9.0,Mid,Least busy,Slowest days,Busy,10,New customer,6.367188,Low,20.5,Low,Female,Alabama,31,2/17/2019,3,married,40423,both,South,Active


## Creating profiling variables

#### Pets (dept_id)
I think this could be accomplished with fewer intermediaries, but not worth spending extra time on right now

In [7]:
# Create a pet owner flag column in the orders table to extract User IDs
# This column will be updated later on the customer level within orders

ords_active_only.loc[(ords_active_only['department_id'] == 8), 'pet_owner'] = 1 
ords_active_only['pet_owner'].fillna(0, inplace= True)

In [8]:
ords_active_only['pet_owner'].value_counts()

0.0    30871504
1.0       93060
Name: pet_owner, dtype: int64

In [13]:
# Create a new table of user IDs w/ pet flag, to use updating our main orders table
pet_owners = ords_active_only.groupby('user_id').agg({'pet_owner': ['max']})

In [10]:
# Create a table of customers with order count and total spend
cust = ords_active_only.groupby('user_id').agg({'order_id': ['count'], 'prices': ['sum']})

In [11]:
# Merge pet owner flag into customer table
cust = cust.merge(pet_owners, on = 'user_id', how = 'left')

In [15]:
# Trim non-pet owners out of the customer pet owners df
cust_pets = cust.loc[cust['pet_owner'].values > 0]

In [16]:
# Add the proper flag to all pet owning user IDs in the main orders table
ords_active_only['pet_owner'] = ords_active_only['user_id'].isin(cust_pets.index)

In [18]:
# Check outcome--it appears normal
ords_active_only['pet_owner'].value_counts()

False    27513213
True      3451351
Name: pet_owner, dtype: int64

In [28]:
# Finally, replace the boolean values with string for later concatenation
ords_active_only.loc[ords_active_only['pet_owner'] == True, 'pet_owner'] = 'pets'
ords_active_only.loc[ords_active_only['pet_owner'] == False, 'pet_owner'] = 'no pets'

#### Age

In [19]:
# Label age groups with flag
ords_active_only.loc[ords_active_only['age'] < 30, 'age_group'] = 'Young Adult'
ords_active_only.loc[(ords_active_only['age'] >= 30) & (ords_active_only['age'] <60), 'age_group'] = 'Adult'
ords_active_only.loc[ords_active_only['age'] >= 60, 'age_group'] = 'Senior'

In [21]:
# Check for any NaN values
ords_active_only['age_group'].value_counts(dropna = False)

Adult          14572457
Senior         10574504
Young Adult     5817603
Name: age_group, dtype: int64

#### Income

In [24]:
# Label income groups with flag
ords_active_only.loc[ords_active_only['income'] < 30000, 'income_group'] = 'low income'
ords_active_only.loc[(ords_active_only['income'] >= 30000) & (ords_active_only['income'] < 125000),
                     'income_group'] = 'mid income'
ords_active_only.loc[ords_active_only['income'] >= 125000, 'income_group'] = 'high income'

In [25]:
ords_active_only['income_group'].value_counts(dropna = False)

mid income     22492650
high income     8278044
low income       193870
Name: income_group, dtype: int64

#### Dependents

In [26]:
# Create parent/non-parent flag
ords_active_only.loc[ords_active_only['n_dependants'] > 0, 'family_size'] = 'parent'
ords_active_only.loc[ords_active_only['n_dependants'] < 1, 'family_size'] = 'non-parent'

In [27]:
ords_active_only['family_size'].value_counts(dropna = False)

parent        23224883
non-parent     7739681
Name: family_size, dtype: int64

### Create customer profiles

In [30]:
# Define a space variable for readability 
a = ' '

In [81]:
# Add flag columns together as completed profiles
ords_active_only['customer_profile'] = ords_active_only['age_group'] + a + ords_active_only['income_group'] +\
a + ords_active_only['family_size']

In [82]:
# View profile quantity IN TERMS OF ORDERS
ords_active_only['customer_profile'].value_counts()

Adult mid income parent               7982831
Senior mid income parent              4668057
Young Adult mid income parent         4250920
Senior high income parent             3252833
Adult high income parent              2873726
Adult mid income non-parent           2637256
Senior mid income non-parent          1545456
Young Adult mid income non-parent     1408130
Senior high income non-parent         1097511
Adult high income non-parent           984998
Adult low income parent                 69108
Young Adult low income parent           64497
Young Adult high income parent          54568
Young Adult low income non-parent       25080
Adult low income non-parent             24538
Young Adult high income non-parent      14408
Senior low income parent                 8343
Senior low income non-parent             2304
Name: customer_profile, dtype: int64

In [85]:
# View profile quantity IN TERMS OF USERIDS
# Group by userid, count the userids in each profile, sort in desc order
(ords_active_only.groupby(ords_active_only['customer_profile'])['user_id'].count()).sort_values\
(ascending = False)

customer_profile
Adult mid income parent               7982831
Senior mid income parent              4668057
Young Adult mid income parent         4250920
Senior high income parent             3252833
Adult high income parent              2873726
Adult mid income non-parent           2637256
Senior mid income non-parent          1545456
Young Adult mid income non-parent     1408130
Senior high income non-parent         1097511
Adult high income non-parent           984998
Adult low income parent                 69108
Young Adult low income parent           64497
Young Adult high income parent          54568
Young Adult low income non-parent       25080
Adult low income non-parent             24538
Young Adult high income non-parent      14408
Senior low income parent                 8343
Senior low income non-parent             2304
Name: user_id, dtype: int64

It appears that both methods of measuring profile quantity agree! Which is nice, since I don't have to decide which would be the more relevant way of determining profile importance

#### Update remaining profiles to Default value

In [70]:
# Create a vector to pass into filter to assign default value
cats = ords_active_only['customer_profile'].value_counts().loc[ords_active_only['customer_profile']
                                                               .value_counts() <2000000].index

In [71]:
cats

Index(['Senior mid income non-parent', 'Young Adult mid income non-parent',
       'Senior high income non-parent', 'Adult high income non-parent',
       'Adult low income parent', 'Young Adult low income parent',
       'Young Adult high income parent', 'Young Adult low income non-parent',
       'Adult low income non-parent', 'Young Adult high income non-parent',
       'Senior low income parent', 'Senior low income non-parent'],
      dtype='object')

In [72]:
# Pass vector into a filter, update anything in the vector to default profile (below 2m)
ords_active_only.loc[ords_active_only['customer_profile'].isin(cats), 'customer_profile'] = 'Default' 

In [74]:
# Results
ords_active_only['customer_profile'].value_counts(dropna = False)

Adult mid income parent          7982831
Default                          5298941
Senior mid income parent         4668057
Young Adult mid income parent    4250920
Senior high income parent        3252833
Adult high income parent         2873726
Adult mid income non-parent      2637256
Name: customer_profile, dtype: int64

#### Descriptive stats - customer profiles

In [78]:
# General order stats per profile group
ords_active_only.groupby('customer_profile').agg({'prices': ['mean', 'median', 'min', 'max']})

prices                  
                                   mean median  min   max
customer_profile                                         
Adult high income parent       7.955380    7.6  1.0  25.0
Adult mid income non-parent    7.744068    7.3  1.0  25.0
Adult mid income parent        7.743009    7.3  1.0  25.0
Default                        7.778623    7.4  1.0  25.0
Senior high income parent      7.956097    7.7  1.0  25.0
Senior mid income parent       7.682342    7.3  1.0  25.0
Young Adult mid income parent  7.804634    7.4  1.0  25.0

In [76]:
# Mode remains the same between all--produce is the most common dept on orders
ords_active_only.groupby(['customer_profile'])['department_id'].agg(pd.Series.mode)

customer_profile
Adult high income parent         4
Adult mid income non-parent      4
Adult mid income parent          4
Default                          4
Senior high income parent        4
Senior mid income parent         4
Young Adult mid income parent    4
Name: department_id, dtype: int8

## Export

In [79]:
ords_active_only.to_pickle(os.path.join(path, '02 Data', 'Processed', 'ords_active_profiles.pkl'))